<a href="https://colab.research.google.com/github/kerengaiger/Deep_Learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Deep Learning EX2** 
Keren Gaiger 305312795 Maria Holodov 317548162

In [12]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Import Data and Libs

In [2]:
!pip install tensorflow==2.0.0-alpha0

  Using cached https://files.pythonhosted.org/packages/9f/4e/402a4d7f21d03cd640b41376727a9ea2bac5a06d2556c04517878dad0c3d/tensorflow-2.0.0a0-cp37-cp37m-manylinux1_x86_64.whl


In [4]:
%load_ext tensorboard.notebook

ModuleNotFoundError: ignored

In [1]:
import numpy as np
import torch as t  
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

ImportError: ignored

In [24]:
%tensorboard --logdir=runs

Launching TensorBoard...

FileNotFoundError: ignored

In [22]:
!pip install torch torchvision

In [3]:
!pip list

Package                       Version             
----------------------------- --------------------
absl-py                       0.12.0              
alabaster                     0.7.12              
albumentations                0.1.12              
altair                        4.1.0               
appdirs                       1.4.4               
argon2-cffi                   20.1.0              
astor                         0.8.1               
astropy                       4.2.1               
astunparse                    1.6.3               
async-generator               1.10                
atari-py                      0.2.6               
atomicwrites                  1.4.0               
attrs                         20.3.0              
audioread                     2.1.9               
autograd                      1.3                 
Babel                         2.9.0               
backcall                      0.2.0               
beautifulsoup4                4

In [ ]:
def data_init():
    with open("/content/drive/MyDrive/deep_learning/ex2_305312795_317548162/ptb.train.txt") as f:
        file = f.read()
        trn = file[1:].split(' ')
    with open("/content/drive/MyDrive/deep_learning/ex2_305312795_317548162/ptb.valid.txt") as f:
        file = f.read()
        vld = file[1:].split(' ')
    with open("/content/drive/MyDrive/deep_learning/ex2_305312795_317548162/ptb.test.txt") as f:
        file = f.read()
        tst = file[1:].split(' ')
    
    words = sorted(set(trn))
    char2ind = {c: i for i, c in enumerate(words)}
    trn = [char2ind[c] for c in trn]
    vld = [char2ind[c] for c in vld]
    tst = [char2ind[c] for c in tst]
    return np.array(trn).reshape(-1, 1), np.array(vld).reshape(-1, 1), np.array(tst).reshape(-1, 1), len(words)

?

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
def minibatch(data, batch_size, seq_length):
    data = t.tensor(data, dtype = t.int64)
    num_batches = data.size(0)//batch_size
    data = data[:num_batches*batch_size]
    data=data.view(batch_size,-1)
    dataset = []
    for i in range(0,data.size(1)-1,seq_length):
        seqlen=int(np.min([seq_length,data.size(1)-1-i]))
        if seqlen<data.size(1)-1-i:
            x=data[:,i:i+seqlen].transpose(1, 0)
            y=data[:,i+1:i+seqlen+1].transpose(1, 0)
            dataset.append((x, y))
    return dataset

# Config Net

In [ ]:
class LstmDrop(nn.Module):
    def __init__(self, vocab_size, hidden_size, layer_num, dropout, winit):
        super().__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        self.winit = winit
        self.lstm_type = lstm_type
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.stacked_lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, 
                             num_layers=layer_num, dropout=dropout)
        self.fc = Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(p=dropout)
        self.reset_parameters()
        
    def reset_parameters(self):
        for param in self.parameters():
            nn.init.uniform_(param, -self.winit, self.winit)
            
    def state_init(self, batch_size, hidden_size, num_layers):
        dev = next(self.parameters()).device
        weight = next(self.parameters()).data
        states = (weight.new_zeros(num_layers, batch_size, hidden_size, device = dev), 
                  weight.new_zeros(num_layers, batch_size, hidden_size, device = dev)) 
        return states
    
    def forward(self, x, states):
        x = self.embed(x)
        x = self.dropout(x)
        output, states = self.stacked_lstm(input, states)
        output = self.dropout(output)
        scores = self.fc(output)
        return scores, states

    def detach(self, states):
        return [(h.detach(), c.detach()) for (h,c) in states]

In [ ]:
def train(train_lst, valid_lst, model, epochs, epoch_threshold, lr, factor, 
          max_norm, epoch_norm):
    total_words = 0
    print("Starting training.\n")
    criterion = nn.CrossEntropyLoss()
    # TODO : test the decrease of lr in factor of 2, initializing it to 1, as suggested in the paper. 
    # TODO : use SGD ?
    opt = t.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.99))

    pbar_train = tqdm.notebook.tqdm(train_lst)
    pbat_valid = tqdm.notebook.tqdm(valid_lst)
    for epoch in range(epochs):
        train_loss, valid_loss, train_prep, valid_prep = 0.0, 0.0, 0.0, 0.0
        
        model.train()
        states = model.state_init(BATCH_SIZE, HIDDEN_SIZE, NUM_LAYERS)

        for x, y in pbar_train:
            model.zero_grad()        
            # Starting each batch, we detach the hidden state from how it was previously produced.
            # If we didn't, the model would try backpropagating all the way to start of the dataset.
            states = model.detach(states)
            scores, states = model(x, states)
            # cross entropy expected to get raw unnormalized and it does the softmax by itself.
            loss = criterion(scores, y)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            t.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
            opt.step()
            train_loss += loss.item()

        with t.no_grad():
            model.eval()
            states = model.state_init(BATCH_SIZE, HIDDEN_SIZE, NUM_LAYERS)
            for x, y in pbar_valid:
                scores, states = model(x, states)
                loss = criterion(scores, y)
                valid_loss += loss.item() 
                states = model.detach(states)
                perp = t.exp(loss)
                valid_prep += prep

        print(f'Epoch : {epoch} || Train loss : {train_loss / len(train_lst)} \
        || Validation loss : {valid_loss / len(valid_lst)} || \
        Validation set perplexity : {perp}')
        print("*************************************************\n")

In [ ]:
#install TensorFlow 2.0 
!pip uninstall tensorflow==2.0.0-alpha0

Uninstalling tensorflow-2.0.0a0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.0.0a0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.0.0a0


In [19]:
# !pip uninstall tensorboard tensorboard-plugin-wit tensorflow-datasets tensorflow-estimator tensorflow-gcs-config tensorflow-hub tensorflow-metadata tensorflow-probability

In [20]:
!pip list

Package                       Version             
----------------------------- --------------------
absl-py                       0.12.0              
alabaster                     0.7.12              
albumentations                0.1.12              
altair                        4.1.0               
appdirs                       1.4.4               
argon2-cffi                   20.1.0              
astor                         0.8.1               
astropy                       4.2.1               
astunparse                    1.6.3               
async-generator               1.10                
atari-py                      0.2.6               
atomicwrites                  1.4.0               
attrs                         20.3.0              
audioread                     2.1.9               
autograd                      1.3                 
Babel                         2.9.0               
backcall                      0.2.0               
beautifulsoup4                4

In [ ]:
writer = SummaryWriter()

In [ ]:
trn, vld, tst, len_w = data_init()

In [ ]:
trn_batches = minibatch(data=trn, batch_size=20, seq_length=20)
vld_batches = minibatch(data=vld, batch_size=20, seq_length=20)

In [ ]:
lstm_drop = LstmDrop(len_w, 200, 2, 0, 0.1)

In [ ]:
lstm_dropout = 